In [1]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.io import push_notebook
from ipywidgets import interact
output_notebook()


Loading BokehJS ...

In [2]:
import pydss

In [60]:
sys1 = pydss.pydss()
sys1.read('lv_europe_connected.json')  # Load data
sys1.read_loads_shapes('load_shapes_list.json')

trafos 0.002182483673095703
lines 0.2697479724884033
trafos2 0.01941370964050293
lines 2 0.35767555236816406
diag_2dsparse 0.11687421798706055
sparse.lil_matrix 0.00019431114196777344
Y_lines_primitive -> Y_primitive_sp 0.19563007354736328
Y_trafos_primitive -> Y_primitive_sp 0.0012502670288085938


/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Ys calc 0.06247448921203613
inv_Y_ii 1.0129399299621582


## CIGRE LV European System

In [61]:
sys1.pf()                            # execute power flow 

In [62]:
pydss.set_load_factor(15*60*60,sys1.params_pf,sys1.params_lshapes)
pydss.pf_eval(sys1.params_pf);

In [63]:
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

In [64]:
%timeit sys1.bokeh_tools()


10 loops, best of 3: 42.2 ms per loop


### Graph with obtained results

In [65]:

p = figure(width=800, height=800,
           title='European LV Network (CIGRE)')#,   x_range=[-200,200], y_range=[-350, 50])

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size='s_radio', color="navy", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

### Interaction with powers

In [66]:
s_0_3pn = sys1.pq_3p # 
s_0_1p  = sys1.pq_1p  # 

p = figure(width=600, height=300,
           title='Voltage vs load powers', 
           x_range = [0,200], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([0,300],[231*1.05,231*1.05], color='red', line_width=5)
p.line([0,300],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update_loads(load_factor=1.0):

    sys1.pq_1p = load_factor*s_0_1p
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [67]:
from ipywidgets import interact
interact(update_loads, load_factor=(-1,1.2, 0.01))

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame()

In [17]:
df['nodes'] = sys1.nodes
df['I_node_m'] = np.abs(sys1.I_node)
df['V_node_m'] = np.abs(sys1.V_node)
df['phi'] = np.angle(sys1.I_node, deg=True) - np.angle(sys1.V_node, deg=True)
df['s'] = np.conjugate(sys1.I_node)*sys1.V_node
df['p'] = df['s'].real
df['q'] = df['s'].imag

In [18]:
self = sys1

node_sorter = []
for bus in self.buses:
    for node in range(10):
        bus_node = '{:s}.{:s}'.format(str(bus['bus']),str(node))
        if bus_node in self.nodes:
            node_idx = self.nodes.index(bus_node) 
            node_sorter += [node_idx]
                
                

In [19]:
sys1.V_node[node_sorter]

array([[  6.35100000e+03   +0.j        ],
       [ -3.17550000e+03-5500.12733943j],
       [ -3.17550000e+03+5500.12733943j],
       ..., 
       [  2.03007975e+02 +118.54050503j],
       [ -2.53434856e-01 -235.31541381j],
       [ -2.05792193e+02 +118.55532689j]])